In [ ]:
# Test deepPET cargando imágenes cargadas directamente con matlab (ficheros mat)
import os
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import tensorflow as tf
import pandas.util.testing as tm
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
tf.test.gpu_device_name()

In [ ]:
import time
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
from skimage import util
import random
import cv2
from tensorflow.keras.models import load_model

from loadDataTomodel import load_data,batch_generator
from  deepPETmodel_128 import deepPETmodel2

In [ ]:

# LOAD YOUR VALIDATION DATA

path='/home/nrufo/Final_final/Data_original/set1/Validation_data/'
fileImage = 'ValidationImages.npy'
fileSinogram = 'ValidationSinograms.npy'

Y_validation = np.load(path + fileImage)
X_validation = np.load(path + fileSinogram)

# ADD NOISE TO SINOGRAMS
noiseFactor = 160
X_validation= X_validation*noiseFactor
X_validation = np.random.poisson(X_validation)

'''
#LOAD YOUR TRAINING DATA THAT IS NOT IN BATCHES
Y_train=np.load('ValidationImages.npy')
X_train=np.load('ValidationSinograms.npy')

X_train= X_train*noiseFactor
X_train = np.random.poisson(X_train)
'''

In [ ]:
# VISUALIZE YOUR DATASET
'''
#if it is in batches
nSet=374
path='/home/nrufo/Final_final/Data/SET_1/Train_data/'

pathx= path+str(nSet)+'_setXtrain.npy'
pathy= path+str(nSet)+'_setYtrain.npy'
X_train = np.load(pathx)
Y_train = np.load(pathy)
'''
nIm=390

Image = np.squeeze(Y_validation[nIm,:,:])
sinogram = np.squeeze(X_validation[nIm,:,:])


plt.subplot(131),plt.imshow(Image, cmap='gray')
plt.subplot(132),plt.imshow(sinogram, cmap='gray')

In [ ]:
# CREATE YOUR MODEL

learning_rate = 0.0001
model = deepPETmodel2(X_validation.shape, learning_rate)
model.summary()

In [ ]:
# TRAIN YOUR MODEL 

history = model.fit(X_train, Y_train, epochs = 125, batch_size = 32, validation_data=(X_validation, Y_validation ),callbacks=[WandbCallback()])
path='Models/Elip1_14_05_Noisy.h5' #path to save your new model
model.save(path)

In [ ]:
# TRAIN YOUR MODEL WITH BATCH DATASET

bs=32
numberBatches= 4535 #
train_generator = batch_generator(numberBatches, batch_size = bs)
history= model.fit(train_generator ,epochs = 125,steps_per_epoch=(numberBatches*bs)//bs,validation_data=(X_validation, Y_validation ),callbacks=[WandbCallback()] )

#callbacks=[WandbCallback()]
# SAVE YOUR MODEL
path='/home/nrufo/Final_final/'
pathfile=path+'06_06_AllCuts.h5' #path to save your new model
model.save(path)

In [ ]:
# SAVE YOUR MODEL
path='/home/nrufo/Final_final/Models/'
nameModel = '18_05_RMS.h5'
pathfile=path+ nameModel #path to save your new model
model.save(path)

In [ ]:
# LOAD MODELS

path='/home/nrufo/Final_final/Models/'
fileModel=path+'18_05_RMS.h5' #path to load your new model
model= load_model(fileModel)  

In [ ]:
# EVALUATE YOUR MODEL

path='/home/nrufo/Final_final/Data_original/set1/Test_data/'

Y_test=np.load(path+'TestImages.npy')
X_test=np.load(path+'TestSinograms.npy')

# Add noise to sinograms
noiseFactor=160
X_test= X_test*noiseFactor
X_test = np.random.poisson(X_test)
# DeepPET reconstruction


preds = model.evaluate(x = X_test, y = Y_test)
print()

print ("Loss = " + str(preds[0]))

print ("Test Accuracy = " + str(preds[1]))

In [ ]:
#VISUALIZE RESULTS

imgID=260
img_predict=model.predict(X_test) 
img_result_train=np.squeeze(img_predict[imgID,:,:])

Image = np.squeeze(np.squeeze(Y_test[imgID,:,:]))
sinogram= np.squeeze(np.squeeze(X_test[imgID,:,:]))

plt.subplot(131), plt.imshow(sinogram, cmap='gray')
plt.title('Original test set'), plt.xticks([]), plt.yticks([])

plt.subplot(132), plt.imshow(Image, cmap='gray')
plt.title('Ground truth'), plt.xticks([]), plt.yticks([])

plt.subplot(133), plt.imshow(img_result_train, cmap='gray')
plt.title('Recon by DeepPET'), plt.xticks([]), plt.yticks([])

plt.show()


In [ ]:
# VISUALIZE YOUR MODEL'S PROGRESS WHILE TRAINING

print(history.history.keys())

# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
tf.keras.backend.clear_session() 